### *Module Loading*

In [ ]:
import sys
from IPython import display

### *External Module Loading*

In [ ]:
external_modules_path = '..\\nn_likelihood_modules'
sys.path.append(external_modules_path)

In [ ]:
from basic_network_structure import *
from common_imports import *
from common_use_functions import *
from constant import *
from defined_data_structure import *
from defined_network_structure import *
from experim_neural_network import *
from experim_preparation import *
from ResNet import *
from experim_ResNet import *
from cifar_10_data_prep import *

### *GPU verification*

In [ ]:
# Get the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nb_gpu = torch.cuda.device_count()
if nb_gpu > 0:
    print(torch.cuda.get_device_name(0))
else:
    print("CPU")

### *Working directory*

In [ ]:
# Current path
current_path = os.path.abspath(os.getcwd())

### *Load configurations and data*

In [ ]:
"""
All the parameters in this part should be configured
"""
# Experience path
experim_path = current_path

# File extensions
json_ext = '.json'
np_ext = '.npy'
csv_ext = '.csv'

# Save paths
model_save_path = path_join(experim_path, 'experim_models_resnet')

# Indices filenames
train_indices_filename = 'train_indices'
valid_indices_filename = 'valid_indices'

# ResNet related params
model_name_prefix = 'cifar10'
resnet_model_name = 'resnet50'

# Tested sets name
train_set_name = 'train'
test_set_name = 'test'
valid_set_name = 'valid'
input_extension = 'X'
label_extension = 'Y'

# Dataset general informations
data_set_infos = {
    'nb_classes' : 10
}

# Batch size for the dataloader creation
torch_batch_size = 128

In [ ]:
# Create the folder to save models and data if not existed
create_directory(model_save_path)

### *Cifar10 dataset preparation*

In [ ]:
# Get the dataset
cifar10_train_dataset, cifar10_test_dataset = get_cifar10_dataset(normalize=False)

In [ ]:
# Train valid split
cifar10_real_train_dataset, cifar10_valid_dataset = torch_random_split(cifar10_train_dataset, split_portion=0.1)

### *ResNet initialization*

In [ ]:
# Create the resnet50
resnet_50 = ResNet50()

In [ ]:
resnet_50

### *ResNet training*

In [ ]:
# Training hyperparameters
lr = 0.0004
optim_type = 'adam'
nb_epochs = 65
criterion_type = 'cross_entropy'

In [ ]:
# Training preparation
train_criterion = get_criterion(criterion_type, mean_reduction=True)
eval_criterion = get_criterion(criterion_type, mean_reduction=False)
optimizer = get_optimizer(resnet_50, optim_type, lr)

In [ ]:
# Dataloader building
train_loader = create_loader_from_torch_dataset(cifar10_real_train_dataset, batch_size=torch_batch_size, shuffle=True, num_workers=0)
valid_loader = create_loader_from_torch_dataset(cifar10_valid_dataset, batch_size=torch_batch_size, shuffle=False, num_workers=0)
test_loader = create_loader_from_torch_dataset(cifar10_test_dataset, batch_size=torch_batch_size, shuffle=False, num_workers=0)

In [ ]:
# Training
train_hist = train_network(resnet_50, nb_epochs, train_loader, valid_loader, optimizer, train_criterion, eval_criterion,
                                    pth=model_save_path, net_name=join_string([model_name_prefix, resnet_model_name]), lr_scheduler=True)

In [ ]:
# Evaluation on the test set
accuracy_eval(resnet_50, test_loader, set_name='test')

In [ ]:
# Save the split indices (can recreate the subset with these indices)
store_list_as_json(path_join(model_save_path,train_indices_filename+'_'+resnet_model_name+json_ext), cifar10_real_train_dataset.indices)
store_list_as_json(path_join(model_save_path,valid_indices_filename+'_'+resnet_model_name+json_ext), cifar10_valid_dataset.indices)